In [ ]:
import matplotlib.pyplot as plt

from report import open_experiment_report, FiveFoldResults, ExperimentReport



In [ ]:
res_2c_mlp = open_experiment_report("../results/exp1_2c_mlp")
res_2c_xgb = open_experiment_report("../results/exp1_2c_xgb")
res_6c_mlp = open_experiment_report("../results/exp1_6c_mlp")
res_6c_xgb = open_experiment_report("../results/exp1_6c_xgb")

In [ ]:
axis_font_size = 14
title_font_size = 16
fig_title_font_size = 18
legend_font_size = 14

# Final results for the table

In [ ]:
def find_best_max(results: list[FiveFoldResults], metric="accuracy"):
    best = (0.0, 0.0, 0) # (max, std, index)
    print(results[0].get_experiment_name())
    for i, res in enumerate(results):
        # print(i)
        max_metric, std = res.get_max(metric)
        final_metric, std = res.get_final(metric)
        if final_metric > best[0]:    
            best = (final_metric, std, i)
        # print(max_metric)

    print(f"    Best {metric} is {best[0]:.4f} ± {best[1]:.4f}")
    print(f"    Final {metric} is {final_metric:.4f}")
    print(f"    {results[best[2]].hyperparameters()}")

find_best_max(res_2c_mlp)
find_best_max(res_6c_mlp)

find_best_max(res_2c_xgb)
find_best_max(res_6c_xgb)

# Overview of figurs for exp. 1

- Overfitting for XGB over the two federations
- MLP training overfitting?

In [ ]:
# Overfitting XGB
indicies= [[1, 5],
           [7, 11]]

fig, ax = plt.subplots(2, 2, figsize=(14, 9))

# Create empty lists to store line objects for the legend
line_2c = []
line_6c = []
line_2c_max = []
line_6c_max = []



for i, j in [[0,0], [0,1], [1,0], [1,1]]:
    run_id = indicies[i][j]
    
    run_2c = res_2c_xgb[run_id]
    run_6c = res_6c_xgb[run_id]
    
    r1, m1, s1 = run_2c.get_mean_std("accuracy")
    r2, m2, s2 = run_6c.get_mean_std("accuracy")
    
    eta1 = run_2c.hyperparameters()["eta"]
    eta2 = run_6c.hyperparameters()["eta"]
    
    md1 = run_2c.hyperparameters()["max_depth"]
    if md1 == 8:
        md1 = 10
    
    max_acc_2c = max(m1)
    max_acc_6c = max(m2)
 
    # Add horizontal lines (store references for legend)
    line1 = ax[i,j].axhline(max_acc_2c, color="blue", linestyle="--")
    line2 = ax[i,j].axhline(max_acc_6c, color="orange", linestyle="--")
    
    # Add data lines (store references for legend)
    line3, = ax[i,j].plot(r1, m1, color="blue")
    line4, = ax[i,j].plot(r2, m2, color="orange")
    
    # Store first occurrence of each line type for the legend
    if i == 0 and j == 0:
        line_2c_max.append(line1)
        line_6c_max.append(line2)
        line_2c.append(line3)
        line_6c.append(line4)
    
    # Set subplot title and properties
    ax[i,j].set_title(f"Validation accuracy, XGB:, Max_depth={md1}, Eta={eta1:.2f} ", 
                      fontsize=title_font_size)
    ax[i,j].set_xlabel("Server rounds", fontsize=axis_font_size)
    ax[i,j].set_ylabel("Accuracy", fontsize=axis_font_size)
    ax[i,j].set_ylim(0.6,0.9)
    ax[i,j].grid()
    

# Create a combined legend
fig.legend(
    [line_2c[0], line_6c[0], line_2c_max[0], line_6c_max[0]],
    ["2 clients", "6 clients", "2 clients max", "6 clients max"],
    loc="lower center", 
    bbox_to_anchor=(0.5, 0.01),
    ncol=4
    , fontsize=legend_font_size
)

# Adjust layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.3, wspace=0.2, bottom=0.15)  # Increased bottom margin for legend

fig.savefig("graphs/exp1_xgb.svg", dpi=300, bbox_inches='tight')


In [ ]:
for i, run in enumerate(res_2c_mlp):
    print(i)
    print(run.hyperparameters())


In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12, 8))

indicies= [[11, 10],
           [9, 8]]

# Create empty lists to store line objects for the legend
line_2c = []
line_6c = []
line_2c_max = []
line_6c_max = []

for i, j in [[0,0], [0,1], [1,0], [1,1]]:
    run_id = indicies[i][j]
    
    run_2c = res_2c_mlp[run_id]
    run_6c = res_6c_mlp[run_id]
    hyperparams = run_2c.hyperparameters()
    lr = hyperparams['lr']
    net = hyperparams['mlp-name']

    print(f"Run {run_id} - {hyperparams}")

    rounds, accuracy_mean2, accuracy_std2 = run_2c.get_mean_std("accuracy")
    rounds, accuracy_mean6, accuracy_std6 = run_6c.get_mean_std("accuracy")
    
    # Find max accuracies
    max_acc_2c = max(accuracy_mean2)
    max_acc_6c = max(accuracy_mean6)
    
    # Add horizontal lines for max values
    line1 = ax[i,j].axhline(max_acc_2c, color="blue", linestyle="--", alpha=0.6)
    line2 = ax[i,j].axhline(max_acc_6c, color="orange", linestyle="--", alpha=0.6)
    
    # Add data lines
    line3, = ax[i,j].plot(rounds, accuracy_mean2, '-', color="blue")
    ax[i,j].fill_between(rounds, accuracy_mean2 - accuracy_std2, accuracy_mean2 + accuracy_std2, alpha=0.1, color="blue")
    
    line4, = ax[i,j].plot(rounds, accuracy_mean6, '-', color="orange")
    ax[i,j].fill_between(rounds, accuracy_mean6 - accuracy_std6, accuracy_mean6 + accuracy_std6, alpha=0.1, color="orange")
    
    # Store first occurrence of each line type for the legend
    if i == 0 and j == 0:
        line_2c_max.append(line1)
        line_6c_max.append(line2)
        line_2c.append(line3)
        line_6c.append(line4)

    ax[i,j].set_title(f"Validation Accuracy, MLP {net}", fontsize=title_font_size)
    ax[i,j].grid(alpha=0.3)
    ax[i,j].set_ylim(0.6, 0.9)
    ax[i,j].set_xlabel("Rounds", fontsize=axis_font_size)
    
    # Add y-axis label only to leftmost plots
    ax[i,j].set_ylabel("Accuracy")

# Create a combined legend
fig.legend(
    [line_2c[0], line_6c[0], line_2c_max[0], line_6c_max[0]],
    ["2 clients", "6 clients", "2 clients max", "6 clients max"],
    loc="lower center", 
    bbox_to_anchor=(0.5, 0.01),
    ncol=4, fontsize=legend_font_size
)

# Add overall title

# Adjust layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.3, wspace=0.2, bottom=0.15, top=0.9)  # Increased bottom margin for legend, top margin for title

fig.savefig("graphs/exp1_mlp_arc.svg", dpi=300, bbox_inches='tight')


# Overfitting based on learning rate

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(14, 5))

indicies = [4, 0, 10]

# Create empty lists to store line objects for the legend
line_2c = []
line_6c = []
line_2c_max = []
line_6c_max = []

for i in range(3):
    run_id = indicies[i]
    
    run_2c = res_2c_mlp[run_id]
    run_6c = res_6c_mlp[run_id]
    hyperparams = run_2c.hyperparameters()
    lr = hyperparams['lr']
    net = hyperparams['mlp-name']

    print(f"Run {run_id} - {hyperparams}")

    rounds, accuracy_mean2, accuracy_std2 = run_2c.get_mean_std("accuracy")
    rounds, accuracy_mean6, accuracy_std6 = run_6c.get_mean_std("accuracy")
    
    # Limit to first 100 rounds if needed
    limit = 100
    if len(rounds) > limit:
        rounds = rounds[:limit]
        accuracy_mean2 = accuracy_mean2[:limit]
        accuracy_std2 = accuracy_std2[:limit]
        accuracy_mean6 = accuracy_mean6[:limit]
        accuracy_std6 = accuracy_std6[:limit]
    
    # Find max accuracies
    max_acc_2c = max(accuracy_mean2)
    max_acc_6c = max(accuracy_mean6)
    
    # Add horizontal lines for max values
    line1 = ax[i].axhline(max_acc_2c, color="blue", linestyle="--", alpha=0.6)
    line2 = ax[i].axhline(max_acc_6c, color="orange", linestyle="--", alpha=0.6)
    
    # Add data lines
    line3, = ax[i].plot(rounds, accuracy_mean2, '-', color="blue")
    ax[i].fill_between(rounds, accuracy_mean2 - accuracy_std2, accuracy_mean2 + accuracy_std2, alpha=0.1, color="blue")
    
    line4, = ax[i].plot(rounds, accuracy_mean6, '-', color="orange")
    ax[i].fill_between(rounds, accuracy_mean6 - accuracy_std6, accuracy_mean6 + accuracy_std6, alpha=0.1, color="orange")
    
    # Store first occurrence of each line type for the legend
    if i == 0:
        line_2c_max.append(line1)
        line_6c_max.append(line2)
        line_2c.append(line3)
        line_6c.append(line4)
    
    ax[i].set_title(f"Validation accuracy, LR={lr}", fontsize=title_font_size)
    ax[i].grid(alpha=0.3)
    ax[i].set_ylim(0.6, 0.9)
    ax[i].set_xlabel("Rounds", fontsize=axis_font_size)

    # Add y-axis label only to leftmost plot
    ax[i].set_ylabel("Accuracy", fontsize=axis_font_size)

# Create a combined legend
fig.legend(
    [line_2c[0], line_6c[0], line_2c_max[0], line_6c_max[0]],
    ["2 clients", "6 clients", "2 clients max", "6 clients max"],
    loc="lower center", 
    bbox_to_anchor=(0.5, 0.01),
    ncol=4,
    fontsize=legend_font_size
)

# Add overall title with learning rate

# Adjust layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.3, wspace=0.3, bottom=0.2)  # Increased bottom margin for legend, top margin for title

fig.savefig("graphs/exp1_mlp_lr.svg", dpi=300, bbox_inches='tight')

In [ ]:
for i, run in enumerate(res_2c_mlp):
    hp = run.hyperparameters()
    hp_important = {k: hp[k] for k in ['mlp-name', 'lr']}
    mean, std = run.get_final("accuracy")
    print(f"Run {i} - {hp_important}, {mean:.4f} ± {std:.4f}")

print("6 clients")
    
for i, run in enumerate(res_6c_mlp):
    hp = run.hyperparameters()
    hp_important = {k: hp[k] for k in ['mlp-name', 'lr']}
    mean, std = run.get_final("accuracy")
    print(f"Run {i} - {hp_important}, {mean:.4f} ± {std:.4f}")

In [ ]:
# XGB results for table
for i, run in enumerate(zip(res_2c_xgb, res_6c_xgb)):
    run_2c, run_6c = run
    hp = run_2c.hyperparameters()
    hp_important = {k: hp[k] for k in ['eta', 'max_depth', 'subsample']}
    mean, std = run_2c.get_final("accuracy")
    mean6, std6 = run_6c.get_final("accuracy")
    print(f"Run {i} - {hp_important}, 2 clients: {mean:.4f} ± {std:.4f}, 6 clients: {mean6:.4f} ± {std6:.4f}")